### Milans experiments with the Twitter API for Berliner Luft project

### Install dependencies

In [33]:
install_packages = False

if install_packages:
    !conda install -c conda-forge python-decouple -y
    !conda install -c conda-forge tweepy -y
    !conda install -c anaconda pandas -y

### Import dependencies

In [34]:
import os
from decouple import config # for loading .env files to securely store api keys
import getpass # to prompt user for access pin
import tweepy # python package to easily access twitter api
import pandas
import webbrowser
import time

### Authorize for API access

In [35]:
# read key and secret from local .env file
consumer_key = config('TWITTER_KEY')
consumer_secret = config('TWITTER_SECRET')

In [36]:
callback_uri = 'oob' # https://cfe.sh/twitter/callback
auth = tweepy.OAuthHandler(consumer_key, consumer_secret, callback_uri)
redirect_url = auth.get_authorization_url()
webbrowser.open(redirect_url)
user_pin_input = getpass.getpass("What's the authorization pin from the website? ")
auth.get_access_token(user_pin_input)
api = tweepy.API(auth, wait_on_rate_limit=True)
me = api.me()
print(me.screen_name)

What's the authorization pin from the website?  ·······


palermo74822531


### Filtering Tweets by location and time
- **Tweets with a specific latitude/longitude "Point" coordinate.** They come from GPS enabled devices, and represent the exact GPS location of the Tweet in question.
- **Tweets with a Twitter “Place”.** They contain a polygon, consisting of 4 lon-lat coordinates that define the general area (the “Place”) from which the user is posting the Tweet. Additionally, the Place will have a display name, type (e.g. city, neighborhood), and country code corresponding to the country where the Place is located, among other fields.
- **Tweets that use the hashtag #friedrichstraße**
- **Get tweets from June 2020 untill today**

In [37]:
search_word = "#friedrichstraße"
location = "52.51275146804908, 13.389556366632663, 0.5km"
date_since = "2020-06-01" #yyyy-mm-dd, unfortunately the twitter api only dates back 1 week but there seem to be ways around this: https://pypi.org/project/GetOldTweets3/


In [58]:
tweets = tweepy.Cursor(api.search,
              q=search_word,
              lang="de",
              since=date_since,
              include_retweets=False,
              tweet_mode='extended').items() # tweet mode extended to get full tweets

tweet_data = []

for tweet in tweets:
    tweet_data.append([tweet.created_at, tweet.user.screen_name, tweet.full_text, tweet.favorite_count, tweet.retweet_count, tweet.coordinates, tweet.user.location, "https://twitter.com/twitter/statuses/" + tweet.id_str])

table = pandas.DataFrame(data=tweet_data, 
                     columns=["timestamp", "user", "text", "likes", "retweets", "coordinates", "location", "url"])
table

# Open table in web browser
html = table.to_html()
path = os.path.abspath('data.html')
url = 'file://' + path
with open(path, 'w') as f:
    f.write(html)
webbrowser.open(url)

True